# Multivariate Poisson using hybrid MM-initial + Newton's for joint estimation

    
   ### Simulation Parameters (Truth):
   $$\mu = 5.000, vc_1 = 0.100, vc_2 = 0.100$$
   
   
   ### QuasiNewton:
    
    1. Using initial MM + Quasi-Newton: (187 iterations, 3661.2 seconds)
   $$\mu = 5.004, vc_1 = 0.110, vc_2 = 0.115$$
    
     
    2. Using initial MM + Quasi-Newton + mehrotra: (331 iterations, 154.3 seconds)
   $$\mu = 5.004, vc_1 = 0.110, vc_2 = 0.115$$
    
   ### Newton:
 
    3. Using initial MM + Newton: (33 iterations, 401.8 seconds)
   $$\mu = 5.004, vc_1 = 0.110, vc_2 = 0.115$$
    
    4. Using initial MM + Newton + mehrotra: (21 iterations, 31.1 seconds)
   $$\mu = 5.004, vc_1 = 0.110, vc_2 = 0.115$$
    
# Summary: Using MM-update on initial beta + using mehrotra algorithm works best (fewest iterations and fastest)

In [1]:
using GLMCopula, Random, Statistics, Test, LinearAlgebra, StatsFuns, GLM, Revise

In [2]:
Random.seed!(1234)
n = 50
variance_component_1 = 0.1
variance_component_2 = 0.1
Γ = variance_component_1 * ones(n, n) + variance_component_2 * Matrix(I, n, n)

mean = 5
dist = Poisson
vecd = [dist(mean) for i in 1:n]
    
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

NonMixedMultivariateDistribution{Discrete,Poisson{Float64},Array{Poisson{Float64},1}}(Poisson{Float64}[Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0)  …  Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0)], [0.2 0.1 … 0.1 0.1; 0.1 0.2 … 0.1 0.1; … ; 0.1 0.1 … 0.2 0.1; 0.1 0.1 … 0.1 0.2], 9.999999999999996, Union{ContinuousUnivariateCopula, DiscreteUnivariateCopula}[#undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef  …  #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef])

In [3]:
nsample = 10_000
Random.seed!(1234)
@time Y_nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, nsample)

  4.605143 seconds (10.14 M allocations: 7.823 GiB, 16.53% gc time)


10000-element Array{Array{Float64,1},1}:
 [7.0, 8.0, 7.0, 4.0, 8.0, 2.0, 6.0, 6.0, 6.0, 7.0  …  5.0, 3.0, 4.0, 6.0, 4.0, 5.0, 5.0, 6.0, 4.0, 7.0]
 [3.0, 7.0, 6.0, 3.0, 6.0, 6.0, 3.0, 3.0, 8.0, 5.0  …  3.0, 4.0, 5.0, 8.0, 2.0, 5.0, 4.0, 4.0, 8.0, 6.0]
 [6.0, 3.0, 3.0, 7.0, 4.0, 3.0, 6.0, 2.0, 1.0, 8.0  …  2.0, 4.0, 3.0, 4.0, 6.0, 3.0, 4.0, 4.0, 3.0, 5.0]
 [9.0, 4.0, 5.0, 3.0, 5.0, 5.0, 4.0, 12.0, 4.0, 5.0  …  4.0, 8.0, 7.0, 9.0, 5.0, 3.0, 8.0, 5.0, 4.0, 4.0]
 [5.0, 7.0, 4.0, 7.0, 4.0, 1.0, 7.0, 3.0, 5.0, 4.0  …  9.0, 7.0, 3.0, 1.0, 3.0, 5.0, 3.0, 6.0, 5.0, 5.0]
 [6.0, 6.0, 10.0, 2.0, 4.0, 2.0, 6.0, 7.0, 1.0, 5.0  …  2.0, 6.0, 3.0, 6.0, 3.0, 6.0, 4.0, 4.0, 4.0, 5.0]
 [5.0, 4.0, 6.0, 3.0, 4.0, 6.0, 0.0, 3.0, 3.0, 3.0  …  4.0, 6.0, 3.0, 3.0, 5.0, 1.0, 4.0, 1.0, 5.0, 10.0]
 [6.0, 4.0, 5.0, 7.0, 5.0, 8.0, 5.0, 3.0, 10.0, 1.0  …  5.0, 4.0, 5.0, 3.0, 4.0, 6.0, 5.0, 4.0, 7.0, 2.0]
 [4.0, 5.0, 8.0, 5.0, 5.0, 4.0, 7.0, 3.0, 3.0, 6.0  …  5.0, 2.0, 4.0, 7.0, 6.0, 5.0, 6.0, 5.0, 7.0, 4.0]
 [4.0, 6.0

In [4]:
d = Poisson()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{GLMCopulaVCObs{T, D, Link}}(undef, nsample)
for i in 1:nsample
    y = Float64.(Y_nsample[i])
    X = ones(n, 1)
    V = [ones(n, n), Matrix(I, n, n)]
    gcs[i] = GLMCopulaVCObs(y, X, V, d, link)
end
gcm = GLMCopulaVCModel(gcs);

initialize_model!(gcm)
@show gcm.β

fill!(gcm.Σ, 1.0)
update_Σ!(gcm)

1 0.0 -1.1106679665667238e6 39999
2 -1.1106679665667238e6 -1.1106679665667238e6 39999
gcm.β = [1.6140126324276531]


2-element Array{Float64,1}:
 0.09215963543599975
 0.09063686128269058

In [5]:
loglikelihood2!(gcm, true, true)

-1.1093040785410176e6

In [6]:
gcm2 = deepcopy(gcm)
gcm3 = deepcopy(gcm)
gcm4 = deepcopy(gcm);

## 1. gcm: Quasi-Newton

In [7]:
@time fit2!(gcm, IpoptSolver(print_level = 5, max_iter = 3000, warm_start_init_point="yes", limited_memory_initialization = "constant", limited_memory_init_val = -gcm.Hβ[1], hessian_approximation = "limited-memory"))

gcm.θ = [1.6140126324276531, 0.09215963543599975, 0.09063686128269058]
gcm.∇θ = [-6095.623635858495, -0.3177125366200926, -0.4550267140366486]
gcm.θ = [1.6140126324276531, 0.09215963543599975, 0.09063686128269058]
gcm.∇θ = [-6095.623635858495, -0.3177125366200926, -0.4550267140366486]

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:  

  29  1.1092928e+06 0.00e+00 3.31e+00  -9.8 5.83e-05    -  1.00e+00 1.70e-313h1040
gcm.θ = [1.6102027237447545, 0.09354222275506695, 0.09195570106948903]
gcm.∇θ = [202.21043398949683, 15.574582651053788, 7.693614290910773]
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  30  1.1092928e+06 0.00e+00 3.32e+00  -9.8 5.83e-05    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.6102043193201618, 0.09357540523405368, 0.09194327408811155]
gcm.∇θ = [198.8739090936925, 12.171588288525527, 10.098642417426163]
  31  1.1092928e+06 0.00e+00 3.26e+00  -9.8 6.64e-05    -  1.00e+00 5.00e-01f  2
gcm.θ = [1.6102094323256446, 0.09366673968308226, 0.0919557551101985]
gcm.∇θ = [188.84601321929946, 5.450576567054668, 14.798974854144035]
  32  1.1092928e+06 0.00e+00 3.10e+00  -9.8 1.83e-04    -  1.00e+00 5.00e-01f  2
gcm.θ = [1.6102188291424342, 0.09383841421807967, 0.09204596269487]
gcm.∇θ = [170.85617473850917, -3.3560204740424524, 20.865764421076165]
  33  1.1092927e+06 0.00e+00 2.80e+

  65  1.1092926e+06 0.00e+00 1.43e+00  -9.8 1.77e-04    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.610220381704214, 0.10610338738746417, 0.10955667159404199]
gcm.∇θ = [87.37677348307376, 5.399274820993014, -0.806737562888884]
  66  1.1092927e+06 0.00e+00 1.43e+00  -9.8 1.14e-04    -  1.00e+00 5.43e-312h1035
gcm.θ = [1.610216959117365, 0.10621520999094304, 0.10967104083646576]
gcm.∇θ = [92.08798328588668, 3.52040743490943, 0.4659333904055529]
  67  1.1092926e+06 0.00e+00 1.51e+00  -9.8 1.14e-04    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.61021279543911, 0.10644558257383223, 0.10992301878804434]
gcm.∇θ = [97.12839917613165, 0.34705232556007104, 2.575249014837027]
  68  1.1092926e+06 0.00e+00 1.59e+00  -9.8 2.52e-04    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.61021279543911, 0.10644558257383223, 0.10992301878804434]
gcm.∇θ = [97.12839917613165, 0.34705232556007104, 2.575249014837027]
  69  1.1092927e+06 0.00e+00 1.59e+00  -9.8 5.18e-04    -  1.00e+00 8.49e-314h1041
gcm.θ = [1.610209980790184, 0.1068875291

 101  1.1092927e+06 0.00e+00 5.85e-03  -9.8 4.05e-07    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.6102597848706357, 0.10977407430469474, 0.11463190140140914]
gcm.∇θ = [0.3951903427262984, 0.024600707700301427, -0.0016761835140641779]
 102  1.1092927e+06 0.00e+00 6.48e-03  -9.8 8.68e-07    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.6102597510808057, 0.10977630119830369, 0.11463440989595573]
gcm.∇θ = [0.43467802931326993, -0.0011745119772967882, 0.015445153891050722]
 103  1.1092927e+06 0.00e+00 7.13e-03  -9.8 2.51e-06    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.6102597510808057, 0.10977630119830369, 0.11463440989595573]
gcm.∇θ = [0.43467802931326993, -0.0011745119772967882, 0.015445153891050722]
 104  1.1092927e+06 0.00e+00 7.13e-03  -9.8 5.50e-06    -  1.00e+00 7.29e-304h1008
gcm.θ = [1.6102597381162134, 0.10978076371813517, 0.11463990555662729]
gcm.∇θ = [0.42621746849339637, -0.034244019276229665, 0.03614224649936659]
 105  1.1092927e+06 0.00e+00 6.99e-03  -9.8 5.50e-06    -  1.00e+00 1.00e+00f  1
gcm.

 136  1.1092927e+06 0.00e+00 1.53e-04  -9.8 3.60e-08    -  1.00e+00 3.82e-298h989
gcm.θ = [1.6102599535185125, 0.1097952663382662, 0.11466125878256955]
gcm.∇θ = [-0.00498529207455789, -0.001111332341679372, 0.0010447065287735313]
 137  1.1092927e+06 0.00e+00 8.18e-05  -9.8 3.60e-08    -  1.00e+00 1.00e+00F  1
gcm.θ = [1.610259953302835, 0.10979524996554776, 0.11466124341195605]
gcm.∇θ = [-0.004487582317723948, -0.0007881313431425951, 0.000824390233901795]
 138  1.1092927e+06 0.00e+00 7.36e-05  -9.8 1.64e-08    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.610259952957145, 0.1097952394779214, 0.11466123268078644]
gcm.∇θ = [-0.0038309637868980673, -0.0006146513624116778, 0.0007082865049596876]
 139  1.1092927e+06 0.00e+00 6.28e-05  -9.8 1.07e-08    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.610259952957145, 0.1097952394779214, 0.11466123268078644]
gcm.∇θ = [-0.0038309637868980673, -0.0006146513624116778, 0.0007082865049596876]
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  

 171  1.1092927e+06 0.00e+00 2.56e-06  -9.8 1.89e-09    -  1.00e+00 5.00e-01f  2
gcm.θ = [1.6102599492860452, 0.10979563227486339, 0.11466180054618966]
gcm.∇θ = [0.00015627774433113473, -1.1119787814095616e-5, 8.528106263838708e-6]
 172  1.1092927e+06 0.00e+00 2.56e-06  -9.8 4.22e-10    -  1.00e+00 1.64e-288h957
gcm.θ = [1.6102599493743337, 0.10979563255040435, 0.1146618009683057]
gcm.∇θ = [8.824574990740075e-6, -1.1046671716030687e-5, 7.916161848697811e-6]
 173  1.1092927e+06 0.00e+00 1.80e-07  -9.8 4.22e-10    -  1.00e+00 1.00e+00F  1
gcm.θ = [1.6102599493789043, 0.10979563238503383, 0.11466180077932787]
gcm.∇θ = [2.4624318282207014e-6, -9.273542673327029e-6, 6.7401197156868875e-6]
 174  1.1092927e+06 0.00e+00 1.51e-07  -9.8 3.78e-10    -  1.00e+00 5.00e-01f  2
gcm.θ = [1.6102599493789043, 0.10979563238503383, 0.11466180077932787]
gcm.∇θ = [2.4624318282207014e-6, -9.273542673327029e-6, 6.7401197156868875e-6]
 175  1.1092927e+06 0.00e+00 1.51e-07  -9.8 5.50e-10    -  1.00e+00 1.68e-28

-1.1092926528694925e6

## 2. gcm2: Quasi-Newton + mehrotra

In [8]:
@time fit2!(gcm2, IpoptSolver(print_level = 5, max_iter = 3000, warm_start_init_point="yes", limited_memory_initialization = "constant", limited_memory_init_val = -gcm2.Hβ[1], mehrotra_algorithm="yes", hessian_approximation = "limited-memory"))

gcm.θ = [1.6140126324276531, 0.09215963543599975, 0.09063686128269058]
gcm.∇θ = [-6095.623635858495, -0.3177125366200926, -0.4550267140366486]
gcm.θ = [1.6140126324276531, 9.99999999, 9.99999999]
gcm.∇θ = [-7000.958205674261, -25.865784293364698, 25.743086909492376]
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        3
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequ

  31  1.1092985e+06 0.00e+00 1.84e-04 -11.0 5.36e-06    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.6098948083596416, 8.327617098817411, 11.664627150108213]
gcm.∇θ = [0.006469777484877426, -0.0075650411101833015, -0.008218614886970954]
  32  1.1092985e+06 0.00e+00 1.35e-04 -11.0 8.23e-06    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.609894776160027, 8.327606840204504, 11.664635384933606]
gcm.∇θ = [0.06215308375445616, -0.00739656542685907, -0.008338459086473904]
  33  1.1092985e+06 0.00e+00 1.02e-03 -11.0 1.03e-05    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.6098947120176181, 8.327599901380022, 11.664639313006735]
gcm.∇θ = [0.1697386151080007, -0.00728813806870221, -0.008415560636137798]
  34  1.1092985e+06 0.00e+00 2.78e-03 -11.0 6.94e-06    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.609894643170417, 8.327603077534613, 11.66463366005473]
gcm.∇θ = [0.2825835456068386, -0.007350936229840954, -0.008370838789062418]
  35  1.1092985e+06 0.00e+00 4.64e-03 -11.0 5.65e-06    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.609894611

  67  1.1092985e+06 0.00e+00 1.84e-01 -11.0 2.65e-04    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.6099012618856572, 8.293216267688702, 11.624942163210394]
gcm.∇θ = [-9.72015099548321, 0.054509930336053, -0.052454517810929246]
  68  1.1092985e+06 0.00e+00 1.59e-01 -11.0 8.87e-04    -  1.00e+00 1.00e+00h  1
gcm.θ = [1.6099004362596394, 8.294041307320647, 11.625887140214521]
gcm.∇θ = [-8.377532878340041, 0.053061018277503536, -0.05142225638928721]
  69  1.1092985e+06 0.00e+00 1.37e-01 -11.0 9.45e-04    -  1.00e+00 1.00e+00h  1
gcm.θ = [1.6098991837180858, 8.29524306207746, 11.627167455102704]
gcm.∇θ = [-6.349432149131589, 0.050259721173006255, -0.04942789107508361]
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  70  1.1092985e+06 0.00e+00 1.04e-01 -11.0 1.28e-03    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.6098973802703802, 8.296626005540988, 11.628401977154946]
gcm.∇θ = [-3.4417384710996863, 0.04535552810699528, -0.045938646374565906]
  71  1.1092985e+06 0.00e+00 5

 103  1.1092984e+06 0.00e+00 4.51e-03 -11.0 1.19e-04    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.6098964844133012, 5.163950899099165, 7.212784104527533]
gcm.∇θ = [1.264979518608527, -0.11693228165847813, 0.048144871600356964]
 104  1.1092984e+06 0.00e+00 2.08e-02 -11.0 2.60e-04    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.609895576480051, 5.163636061917707, 7.2121390651576345]
gcm.∇θ = [2.7333597310684787, -0.12053718184290574, 0.050709689083191734]
 105  1.1092984e+06 0.00e+00 4.48e-02 -11.0 6.45e-04    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.609894448370867, 5.162915455710381, 7.210696719783762]
gcm.∇θ = [4.52823353906496, -0.12838136796347044, 0.0562891276568962]
 106  1.1092984e+06 0.00e+00 7.43e-02 -11.0 1.44e-03    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.6098935515461477, 5.161443917801415, 7.2078023359378465]
gcm.∇θ = [5.874458438377548, -0.14379573782491661, 0.06725181482659082]
 107  1.1092984e+06 0.00e+00 9.64e-02 -11.0 2.89e-03    -  1.00e+00 1.00e+00h  1
gcm.θ = [1.6098940759684894, 5.1593103

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 140  1.1093025e+06 0.00e+00 1.64e+00 -10.5 5.11e+00    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.609862299519139, 3.7009945656326004, 5.188616203675295]
gcm.∇θ = [70.21699453675521, 0.9544286071350143, -0.742532537949046]
 141  1.1092985e+06 0.00e+00 1.15e+00 -11.0 1.42e-02    -  9.90e-01 1.00e+00f  1
gcm.θ = [1.6099163177919449, 3.6884372823221256, 5.142209304228899]
gcm.∇θ = [-25.936688325953646, -0.11272041895313123, 0.012011217732151636]
 142  1.1092989e+06 0.00e+00 4.25e-01 -11.0 4.64e-02    -  9.95e-01 1.00e+00h  1
gcm.θ = [1.60991502512145, 3.6886311629844033, 5.143038415683626]
gcm.∇θ = [-23.666649094972747, -0.09176162304402169, -0.002889409785893804]
 143  1.1092983e+06 0.00e+00 3.88e-01 -11.0 8.29e-04    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.6099134586359198, 3.688690721897958, 5.1435819519974455]
gcm.∇θ = [-20.965855376950497, -0.07428794236648173, -0.015311907099430755]
 144  1.1092983e+06 0.00e+00 3

...

 212  1.1092930e+06 0.00e+00 1.02e+01 -11.0 4.86e-03    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.610057055313732, 0.07469783631596152, 0.06871777456491737]
gcm.∇θ = [654.0206318307871, 325.2367283327185, -153.94662547237385]
 213  1.1092936e+06 0.00e+00 1.07e+01 -11.0 8.83e-03    -  1.00e+00 1.00e+00h  1
gcm.θ = [1.6102268283390402, 0.0824505827854774, 0.0805600717980925]
gcm.∇θ = [292.34265789593337, 323.8414907018596, -183.88559269897814]
 214  1.1092953e+06 0.00e+00 5.31e+00 -11.0 1.18e-02    -  1.00e+00 1.00e+00h  1
gcm.θ = [1.6104680672285314, 0.08884560271212763, 0.09012926161967692]
gcm.∇θ = [-165.418286854889, 305.98449514750246, -188.4666161952191]
 215  1.1092956e+06 0.00e+00 5.02e+00 -11.0 9.57e-03    -  1.00e+00 1.00e+00h  1
gcm.θ = [1.6105673930791402, 0.09159559207039146, 0.09423855109646494]
gcm.∇θ = [-352.5730813057026, 298.6575992579892, -188.9053320276871]
 216  1.1092943e+06 0.00e+00 5.78e+00 -11.0 4.11e-03    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.6105827261591634, 0.092490

 248  1.1092922e+06 0.00e+00 4.95e-01 -11.0 1.34e-03    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.6102620647642796, 0.10944431166678098, 0.11409957309258853]
gcm.∇θ = [-1.9785573463152133, -2.7858382186168456, 2.2525685566215543]
 249  1.1092925e+06 0.00e+00 4.57e-02 -11.0 1.62e-04    -  1.00e+00 1.00e+00h  1
gcm.θ = [1.6102594694958727, 0.10939914634251498, 0.11405326994998405]
gcm.∇θ = [2.681997588197845, -2.0225895467956434, 1.7457360601983964]
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 250  1.1092926e+06 0.00e+00 4.40e-02 -11.0 4.63e-05    -  1.00e+00 1.00e+00h  1
gcm.θ = [1.6102594141692668, 0.10938251304236692, 0.11403473156873893]
gcm.∇θ = [2.8984022996449905, -1.8163845196249895, 1.6093997355783358]
 251  1.1092927e+06 0.00e+00 4.75e-02 -11.0 1.85e-05    -  1.00e+00 1.00e+00h  1
gcm.θ = [1.610259440708083, 0.10937852317777123, 0.11403032670493894]
gcm.∇θ = [2.884695329720728, -1.7657424742064904, 1.5756840275134891]
 252  1.1092927e+06 0.00e+00 

 284  1.1092927e+06 0.00e+00 2.79e-03 -11.0 5.18e-07    -  1.00e+00 1.00e+00h  1
gcm.θ = [1.6102599128781092, 0.10979176565130433, 0.11465663584527143]
gcm.∇θ = [0.08373679548863322, 0.01268514696857892, -0.006233864449313842]
 285  1.1092927e+06 0.00e+00 1.37e-03 -11.0 1.96e-06    -  1.00e+00 1.00e+00h  1
gcm.θ = [1.6102599604885632, 0.10979356249632377, 0.1146591387117567]
gcm.∇θ = [-0.00524224957442776, 0.010582930590317119, -0.0061953424592502415]
 286  1.1092927e+06 0.00e+00 1.74e-04 -11.0 2.50e-06    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.6102599751085158, 0.10979451554281074, 0.11466028535128701]
gcm.∇θ = [-0.03600526398257031, 0.0021492720989044756, -0.0008580380672356114]
 287  1.1092927e+06 0.00e+00 5.91e-04 -11.0 1.15e-06    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.6102599735794114, 0.10979465490591458, 0.11466039592938602]
gcm.∇θ = [-0.03480272808008955, -0.0013894961812184192, 0.0015986466023605317]
 288  1.1092927e+06 0.00e+00 5.71e-04 -11.0 1.39e-07    -  1.00e+00 1.00e+00f  1
g

 319  1.1092927e+06 0.00e+00 2.55e-07 -11.0 7.46e-10    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.6102599493751022, 0.10979563129070687, 0.11466179924610889]
gcm.∇θ = [1.4892415988398966e-5, -8.731272828299552e-6, 7.1442045737057924e-6]
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 320  1.1092927e+06 0.00e+00 2.44e-07 -11.0 1.35e-09    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.6102599493659984, 0.10979563205756616, 0.11466180045409904]
gcm.∇θ = [2.652449655293765e-5, -3.420216671656817e-6, 2.8070571715943515e-6]
 321  1.1092927e+06 0.00e+00 4.35e-07 -11.0 1.21e-09    -  1.00e+00 1.00e+00h  1
gcm.θ = [1.6102599493699015, 0.10979563222422166, 0.11466180079252285]
gcm.∇θ = [1.9806959084434084e-5, 7.843336788937449e-7, -3.5719131119549274e-7]
 322  1.1092927e+06 0.00e+00 3.25e-07 -11.0 3.38e-10    -  1.00e+00 1.00e+00h  1
gcm.θ = [1.6102599493733427, 0.10979563216967746, 0.11466180074114041]
gcm.∇θ = [1.4578263954234671e-5, 1.7921338439919055e-6, -1.0596340871416032

-1.1092926528694874e6

## 3. gcm3: Just Newton (no merotra)

In [9]:
@time fit2!(gcm3, IpoptSolver(print_level = 5, max_iter = 100, hessian_approximation = "exact"))

gcm.θ = [1.6140126324276531, 0.09215963543599975, 0.09063686128269058]
gcm.∇θ = [-6095.623635858495, -0.3177125366200926, -0.4550267140366486]
gcm.θ = [1.6140126324276531, 0.09215963543599975, 0.09063686128269058]
gcm.∇θ = [-6095.623635858495, -0.3177125366200926, -0.4550267140366486]
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        4

Total number of variables............................:        3
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  30  1.1092927e+06 0.00e+00 4.10e-08  -8.6 1.86e-10    -  1.00e+00 1.00e+00F  1
gcm.θ = [1.6102599493656469, 0.10979563630877397, 0.11466180647466803]
gcm.∇θ = [3.682825868267514e-6, -2.6470734155736864e-6, -7.787530194036663e-7]
  31  1.1092927e+06 0.00e+00 6.04e-08  -8.6 1.32e-09    -  1.00e+00 5.00e-01f  2
gcm.θ = [1.6102599493678662, 0.10979563581569224, 0.1146618058089122]
gcm.∇θ = [2.9362804454535762e-6, -1.4241813914850354e-6, -1.318250724580139e-6]
  32  1.1092927e+06 0.00e+00 4.82e-08  -8.6 6.66e-10    -  1.00e+00 1.00e+00F  1
gcm.θ = [1.6102599493696357, 0.10979563580191896, 0.11466180579026428]
gcm.∇θ = [8.860680900113493e-8, -1.4165343600325286e-6, -1.3200988178141415e-6]
  33  1.1092927e+06 0.00e+00 1.45e-09  -8.6 1.86e-11    -  1.00e+00 1.00e+00F  1

Number of Iterations....: 33

                                   (scaled)                 (unscaled)
Objective...............:   1.819818150096

-1.1092926528694867e6

## 4. gcm4: Newton using Hessian + merotra

In [10]:
@time fit2!(gcm4, IpoptSolver(print_level = 5, max_iter = 100, mehrotra_algorithm="yes", warm_start_init_point="yes", hessian_approximation = "exact"))

gcm.θ = [1.6140126324276531, 0.09215963543599975, 0.09063686128269058]
gcm.∇θ = [-6095.623635858495, -0.3177125366200926, -0.4550267140366486]
gcm.θ = [1.6140126324276531, 9.99999999, 9.99999999]
gcm.∇θ = [-7000.958205674261, -25.865784293364698, 25.743086909492376]
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        4

Total number of variables............................:        3
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequ

-1.1092926528694904e6

## Loglikelihoods match up to 13th digit 

In [11]:
@show GLMCopula.loglikelihood!(gcm, true, true) 
@show GLMCopula.loglikelihood!(gcm2, true, true) 
@show GLMCopula.loglikelihood!(gcm3, true, true) 
@show GLMCopula.loglikelihood!(gcm4, true, true);

GLMCopula.loglikelihood!(gcm, true, true) = -1.1092926528694925e6
GLMCopula.loglikelihood!(gcm2, true, true) = -1.1092926528694874e6
GLMCopula.loglikelihood!(gcm3, true, true) = -1.1092926528694867e6
GLMCopula.loglikelihood!(gcm4, true, true) = -1.1092926528694904e6


## Mean match to 10th digit, variance components match until the 9th digit

In [12]:
@show gcm.θ
@show gcm2.θ
@show gcm3.θ
@show gcm4.θ;

gcm.θ = [1.610259949385557, 0.1097956327283411, 0.11466180146992966]
gcm2.θ = [1.6102599493808403, 0.10979563244428904, 0.1146618010784021]
gcm3.θ = [1.6102599493696357, 0.10979563580191896, 0.11466180579026428]
gcm4.θ = [1.6102599493811418, 0.10979563246091144, 0.11466180111186794]


## Newton with mehrotra seems to be solving the best

In [13]:
@show gcm.∇θ
@show gcm2.∇θ
@show gcm3.∇θ
@show gcm4.∇θ;

gcm.∇θ = [-9.120569174569937e-6, -8.636779966764152e-7, 4.0639861254021525e-7]
gcm2.∇θ = [3.2118917481227527e-7, -4.018584149889648e-7, 2.952729118987918e-7]
gcm3.∇θ = [8.860680900113493e-8, -1.4165343600325286e-6, -1.3200988178141415e-6]
gcm4.∇θ = [-2.053773435761741e-7, 7.2177499532699585e-9, -1.1939846444875002e-8]


# New Results: Hybrid MM-initial + Newton's for joint estimation

   ### QuasiNewton:
    
    1. Using initial MM + Quasi-Newton:
    5.004 0.110 0.115 (187 iterations, 3661.2 seconds)
    
     
    2. Using initial MM + Quasi-Newton + mehrotra:
    5.004 0.110 0.115 (331 iterations, 154.3 seconds)
    
   ### Newton:
 
    3. Using initial MM + Newton:
    5.004 0.110 0.115 (33 iterations, 401.8 seconds)
    
    4. Using initial MM + Newton + mehrotra:
    5.004 0.110 0.115 (21 iterations, 31.1 seconds)
    

# Old Results: Hybrid MM-initial + Newton's for joint estimation

   ### Quasi-Newton:
    
    1. Using the MM-update + Quasi-Newton:
    5.004  0.110  0.115 (863 iterations, 994.3 seconds)
    
    2. Using the MM-update + Quasi-Newton + mehrotra:
    5.004  0.110  0.115 (2843 iterations, 2018.5 seconds)
    
   ### Newton:
    3. Using the MM-update + Newton:
    5.004  0.110  0.115 (1617 iterations, 3051.4 seconds)

    4. Using the MM-update + Newton + mehrotra:
    5.004  0.110  0.115 (1784 iterations, 3137.7 seconds)